Introduction to TensorFlow (Follow Along)
===

We are going to learn the basics of TensorFlow by it's Python API. We will cover the following sections:
1. Tensors
2. TensorFlow Core (low-level API)
3. tf.estimator (high-level API)

**Note:** This document is part of the "Introduction to TensorFlow" workshop, which is meant to accompany these [slides](https://docs.google.com/presentation/d/1mJSo6hLstwFxo9-R68mZKQVGC6uaVBl-jx5Yq13JUyA/edit?usp=sharing). The main source for this notebook is https://www.tensorflow.org/get_started/get_started. For more information please visit [AI for Everyone!](http://andresvourakis.com/ai-for-everyone)

Tensors
---

The central unit of data in TensorFlow is a **tensor**. A tensor consists of a set of primitive values shaped into array of any number of dimensions. A tensor's **rank** is its number of dimesions.

In [5]:
# A rank 0 tensor; a scalar with shape []
3

3

In [7]:
# A rank 1 tensor; a scalar with shape [3]
[2, 3, 3]

[2, 3, 3]

In [8]:
# A rank 2 tensor; a matrix with shape [2, 3]
[[2,3,4],[1,2,3]]

[[2, 3, 4], [1, 2, 3]]

TensorFlow Core (low-level API)
---

**Importing TensorFlow**   
In order to give Python access to all of TensorFlow's classes, methods, and symbols we need to import it as follows:

In [9]:
# The canonical import statement for TensorFlow programs
import tensorflow as tf

**The Computational Graph**   
TensorFlow Core programs consist of two discrete sections:
1. Building the computational graph 
2. Running the computational graph

A computational graph is a series of TensorFlow operations arranged into a graph of nodes. Each node takes zero or more tensors as inputs and produces a tensor as output. Let's build a computational graph using a **constant**:

**tf.constant**  
It takes no inputs, and it outputs a value it stores internally.

In [14]:
# Building the computational graph
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(1.0)

node3 = tf.add(node1,node2)

print(node1)
print(node2)
print(node3)

Tensor("Const_6:0", shape=(), dtype=float32)
Tensor("Const_7:0", shape=(), dtype=float32)
Tensor("Add_1:0", shape=(), dtype=float32)


To actually evaluate the nodes, we must run the computational graph within a session. A session encapsulates the control and state of the TensorFlow runtime.

In [15]:
# Running the computatinal graph
sess = tf.Session()

print(node1)
print(sess.run(node1))
print(sess.run(node3))

Tensor("Const_6:0", shape=(), dtype=float32)
3.0
4.0


**Note: **TensorFlow provides a utility called TensorBoard that can display a picture of the computational graph. Here is an example: ![addition operation](https://www.tensorflow.org/images/getting_started_add.png)

**tf.placeholder**  
A placeholder is a promise to provide a value later

In [25]:
# Building the computational graph
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = tf.add(a, b)


# Running the computatinal graph
sess = tf.Session()
print(sess.run(adder_node, {a: [2,3,4], b: [1,2,2]}))

[ 3.  5.  6.]


**tf.Variable**  
A variable is contructed with a type and initial value that be changed later.

In [34]:
# Building the computational graph
a = tf.Variable(3.0, dtype=tf.float32)
b = tf.Variable(2.0, dtype=tf.float32)
adder_node = a + b
fixa = tf.assign(a, adder_node)
fixb = tf.assign(b, adder_node)

# Don't forget to Initialize Variables
init = tf.global_variables_initializer()

# Running the computatinal graph
sess = tf.Session()
sess.run(init)
sess.run(fixa)
print(sess.run(a))

5.0


**tf.assign**

**tf.train** API  
Provides a set of classes and functions that help train models. Whitin this set of classes and functions we have optimizers that slowly change each variable in order to minimize the loss function.

In [29]:
# Lets optimize using gradient descent
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minize(loss) # Define loss first

**Trainable Linear Regression Model**  
Lets build a complete trainable linear regression model

In [49]:
# Imports
import tensorflow as tf

# Model parameters
W = tf.Variable([0.1], dtype=tf.float32)
b = tf.Variable([-0.1], dtype=tf.float32)
init = tf.global_variables_initializer()

# Model input and output
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
linear_model = W*x + b

#loss
loss = tf.reduce_sum(tf.square(linear_model - y)) #sum of squares

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1,2,3,4]
y_train = [0, -1 , -2, -3]

# training loop
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        sess.run(train, {x: x_train, y: y_train})


    # evaluate training accuracy
    curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
    print('Current Weights: {}, Current Bias: {}, Current Loss: {}'.format(curr_W, curr_b, curr_loss))

Current Weights: [-0.99999744], Current Bias: [ 0.99999249], Current Loss: 3.835864959000901e-11


tf.estimator (high-level API)
---

**tf.estimator** is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:
* running training loops
* running evaluating loops
* anaging data sets

Let's use tf.estimator to build a complete trainable linear regression model

In [48]:
# Imports
import numpy as np
import tensorflow as tf

# Declare list of features
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# Declare linear regression estimator
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# Read and set up data sets
x_train = np.array([1, 2, 3, 4])
y_train = np.array([0, -1, -2, -3])
input_fn = tf.estimator.inputs.numpy_input_fn({'x': x_train}, y_train, batch_size = 4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn({'x': x_train}, y_train, batch_size = 4, num_epochs=1000, shuffle=False)

# Train model
estimator.train(input_fn=input_fn, steps=1000)

# Evaluate how well our model did
train_metrics = estimator.evaluate(input_fn=train_input_fn)
print(train_metrics)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\avour\\AppData\\Local\\Temp\\tmpwvwc7996', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D707AE7828>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\avour\\AppData\\Local\\Temp\\tmpwvwc7996', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D707AE7828>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Finished evaluation at 2017-11-14-00:06:20


INFO:tensorflow:Finished evaluation at 2017-11-14-00:06:20


INFO:tensorflow:Saving dict for global step 1000: average_loss = 3.0164e-09, global_step = 1000, loss = 1.20656e-08


INFO:tensorflow:Saving dict for global step 1000: average_loss = 3.0164e-09, global_step = 1000, loss = 1.20656e-08


{'average_loss': 3.0163965e-09, 'loss': 1.2065586e-08, 'global_step': 1000}


**Note: ** tf.estimator does not lock you into its predefined models. Suppose we wanted to create a custom model that is not built into TensorFlow. We can still retain the high level abstraction of data set, feeding, training, etc. of tf.estimator